@Joshua Ong

In [1]:
using Pkg
Pkg.add("Pkg")
Pkg.add("Dates")

joshpath = "C:/Users/Owner/Documents/Austin/AmbulanceDeployment/AmbulanceDeployment.jl-legacy"

PROJECT_ROOT = joshpath
currentpath = joshpath

cd(currentpath)
Pkg.activate(".")

#include(".//src//problem.jl")
#include(".//src//model.jl")

using AmbulanceDeployment 
using DataFrames, Winston, JLD, CSV, Gurobi, JuMP, GLPK, Dates

   Updating registry at `C:\Users\Owner\.julia\registries\General`
  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`
 Activating environment at `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
┌ Info: Precompiling AmbulanceDeployment [d2c08a37-75f3-4934-bcbc-64feba624d07]
└ @ Base loading.jl:1278


In [8]:
adjacent_nbhd = CSV.File("C:/Users/Owner/Documents/Austin/AmbulanceDeployment/AmbulanceDeployment.jl-legacy/test/austin_data/adjacent_nbhd.csv") |> DataFrame
coverage = CSV.File("C:/Users/Owner/Documents/Austin/AmbulanceDeployment/AmbulanceDeployment.jl-legacy/test/austin_data/coverage.csv") |> DataFrame
hourly_calls = CSV.File("C:/Users/Owner/Documents/Austin/AmbulanceDeployment/AmbulanceDeployment.jl-legacy/test/austin_data/Full_WeekdayCalls.csv") |> DataFrame

print("\n coverage ",size(coverage))
print("\n hourly_calls ",size(hourly_calls))
print("\n adjacent_nbhd ",size(adjacent_nbhd))

num_regions = 209
regions = collect(1:209)

#reformat data frames as matrix data
locations = collect(1:size(coverage,2))
coverage = Matrix(coverage)
coverage = convert(Array{Bool, 2}, coverage[:, :])
adjacent = Matrix(adjacent_nbhd[!,2:end])
adjacent = adjacent[regions,regions] .> 0.5
demand = Matrix(hourly_calls[!,5:end-1]); #we were trimming first 6 collumns but it seems to me it only the first 5 contain extraneous data

print("\n coverage ",size(coverage))
print("\n demand ",size(demand))
print("\n adjacent ",size(adjacent))


 coverage (209, 44)
 hourly_calls (10296, 214)
 adjacent_nbhd (224, 225)
 coverage (209, 44)
 demand (10296, 209)
 adjacent (209, 209)

In [9]:
# We focus on emergency calls during the "peak period" (8AM - 8PM),
# with the emergency calls from the first 3 month as our training set,
# and the subsequent emergency calls from the remaining months as our test set
peak_period = (hourly_calls[!,:hour] .>= 8) .* (hourly_calls[!,:hour] .<= 20)
indices = 1:DataFrames.nrow(hourly_calls);
train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
test_filter  = .~train_filter;
train_indices = indices[train_filter]
test_indices = indices[test_filter];

# we distinguish between peak and offpeak hours
train_peak = indices[peak_period .* train_filter]
train_offpeak = indices[.~peak_period .* train_filter]

test_peak = indices[peak_period .* test_filter]
test_offpeak = indices[.~peak_period .* test_filter]

p = DeploymentProblem(30, length(locations), length(regions), demand, train_indices,
      test_indices, coverage[regions,:], Array{Bool,2}(adjacent));

In [11]:
namb = 50
alpha = .1
#data structures to record results
scenarios = Dict{Symbol, Dict{Int, Vector{Vector{Int}}}}()
generated_deployment = Dict{Symbol, Dict{Int, Vector{Vector{Int}}}}()
upperbounds = Dict{Symbol, Dict{Int, Vector{Float64}}}()
lowerbounds = Dict{Symbol, Dict{Int, Vector{Float64}}}()
upptiming = Dict{Symbol, Dict{Int, Vector{Float64}}}()
lowtiming = Dict{Symbol, Dict{Int, Vector{Float64}}}()
amb_deployment = Dict{Symbol, Dict{Int, Vector{Int}}}()

(deployment_model, name) = (next_dp -> RobustDeployment(next_dp, α=alpha), :Robust01)
amb_deployment[name] = Dict{Int, Vector{Int}}()
scenarios[name] = Dict{Int, Vector{Vector{Int}}}()
generated_deployment[name] = Dict{Int, Vector{Vector{Int}}}()
upperbounds[name] = Dict{Int, Vector{Float64}}()
lowerbounds[name] = Dict{Int, Vector{Float64}}()
upptiming[name] = Dict{Int, Vector{Float64}}()
lowtiming[name] = Dict{Int, Vector{Float64}}()
println("$namb ")
p.nambulances = namb

50 


50

In [19]:
sum(model.scenarios[3])

63

In [12]:
model = deployment_model(p)
set_optimizer(model.m, Gurobi.Optimizer)
@time AmbulanceDeployment.optimize_robust!(model, p);

amb_deployment[name][namb] = deployment(model)
# for tracking purposes
scenarios[name][namb] = model.scenarios
generated_deployment[name][namb] = model.deployment
upperbounds[name][namb] = model.upperbounds
lowerbounds[name][namb] = model.lowerbounds
upptiming[name][namb] = model.upptiming
lowtiming[name][namb] = model.lowtiming

Academic license - for non-commercial use only - expires 2022-07-26


┌ Warning: Passing optimizer attributes as keyword arguments to
│ Gurobi.Optimizer is deprecated. Use
│     MOI.set(model, MOI.RawParameter("key"), value)
│ or
│     JuMP.set_optimizer_attribute(model, "key", value)
│ instead.
└ @ Gurobi C:\Users\Owner\.julia\packages\Gurobi\BAtIN\src\MOI_wrapper\MOI_wrapper.jl:273


Academic license - for non-commercial use only - expires 2022-07-26


┌ Warning: Passing optimizer attributes as keyword arguments to
│ Gurobi.Optimizer is deprecated. Use
│     MOI.set(model, MOI.RawParameter("key"), value)
│ or
│     JuMP.set_optimizer_attribute(model, "key", value)
│ instead.
└ @ Gurobi C:\Users\Owner\.julia\packages\Gurobi\BAtIN\src\MOI_wrapper\MOI_wrapper.jl:273


gamma global 63gamma global AmbulanceDeployment.robustGamma([1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 4, 7, 5, 3, 4, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 4, 6, 6, 6, 8, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 11, 8, 3, 3, 2, 1, 1, 1, 1, 1, 2, 3, 2, 2, 2, 4, 8, 6, 1, 1, 1, 1, 1, 1, 2, 3, 4, 2, 3, 2, 6, 8, 3, 5, 3, 2, 2, 1, 1, 1, 1, 2, 1, 3, 1, 5, 6, 4, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 3, 5, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 5, 5, 2, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 3, 8, 12, 11, 7, 2, 2, 1, 0, 0, 0, 0, 0, 0, 1, 1, 6, 19, 18, 14, 7, 4, 1, 0, 0, 0, 0, 0, 1, 1, 1, 2, 14, 22, 20, 10, 4, 1, 1, 0, 0, 0, 0, 2, 2, 2, 2, 4, 16, 19, 12, 

Float64[]

In [13]:
solver_stats = Dict{String, Any}()
push!(solver_stats, "amb_deployment" => amb_deployment)
push!(solver_stats, "scenarios" => scenarios)
push!(solver_stats, "generated_deployment" => generated_deployment)
push!(solver_stats, "upperbounds" => upperbounds)
push!(solver_stats, "lowerbounds" => lowerbounds)
push!(solver_stats, "upptiming" => upptiming)
push!(solver_stats, "lowtiming" => lowtiming)
json_string = JSON.json(solver_stats)

open(string(PROJECT_ROOT , "/src/outputs/solver_stats_8_7.json"),"w") do f
 write(f, json_string)
end

LoadError: [91mUndefVarError: JSON not defined[39m

# warning

In [2]:
# hourly calls - regions x hours x number of calls/ per region per hour (e.g. 210 x 10000 x Z)
#adjacent_nbhd - boolean matrix, regionxregion (210x210) - if neighborhoods/region are adjacent?
#coverage - boolean matrix, regions x stations (210x44) - true if station is within coverage of region
#incidents - god object

# PROJECT_ROOT = "C:/Users/Owner/Documents/Austin/AmbulanceDeployment/AmbulanceDeployment.jl-legacy"
# adjacent_nbhd = CSV.File(PROJECT_ROOT * "/test/austin-data/adjacent_nbhd.csv") |> DataFrame
# #I CAN'T for the life of me see the difference.
adjacent_nbhd = CSV.File("C:/Users/Owner/Documents/Austin/AmbulanceDeployment/AmbulanceDeployment.jl-legacy/test/austin_data/adjacent_nbhd.csv") |> DataFrame
coverage = CSV.File("C:/Users/Owner/Documents/Austin/AmbulanceDeployment/AmbulanceDeployment.jl-legacy/test/austin_data/coverage.csv") |> DataFrame
#coverage = convert(Array{Bool, 2}, coverage[:, :])
hourly_calls = CSV.File("C:/Users/Owner/Documents/Austin/AmbulanceDeployment/AmbulanceDeployment.jl-legacy/test/austin_data/Full_WeekdayCalls.csv") |> DataFrame
# weekend_hourly_calls = CSV.File("data/processed/2-weekend_calls.csv") |> DataFrame

print("\n coverage ",size(coverage))
print("\n hourly_calls ",size(hourly_calls))
print("\n adjacent_nbhd ",size(adjacent_nbhd))

num_regions = 209
regions = collect(1:209)

#reformat data frames as matrix data
#regions - list of regions/neighborhoods where demands come from (1 ... 210)
#locations -list of stations (1 ... 44)
#demands - hours x regions = demands
#regions = Int[parse(Int,string(x)) for x in names(hourly_calls[:,6:ncol(hourly_calls)])]
locations = collect(1:size(coverage,2))
coverage = Matrix(coverage)
coverage = convert(Array{Bool, 2}, coverage[:, :])
adjacent = Matrix(adjacent_nbhd[!,2:end]) 
adjacent = adjacent[regions,regions] .> 0.5
demand = Matrix(hourly_calls[!,5:end-1]); #we were trimming first 6 collumns but it seems to me it only the first 5 contain extraneous data

print("\n coverage ",size(coverage))
print("\n demand ",size(demand))
print("\n adjacent ",size(adjacent))


 coverage (209, 44)
 hourly_calls (10296, 214)
 adjacent_nbhd (224, 225)
 coverage (209, 44)
 demand (10296, 209)
 adjacent (209, 209)

In [3]:
# We focus on emergency calls during the "peak period" (8AM - 8PM),
# with the emergency calls from the first 3 month as our training set,
# and the subsequent emergency calls from the remaining months as our test set
peak_period = (hourly_calls[!,:hour] .>= 8) .* (hourly_calls[!,:hour] .<= 20)
indices = 1:DataFrames.nrow(hourly_calls);
train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
test_filter  = .~train_filter;
train_indices = indices[train_filter]
test_indices = indices[test_filter];

In [4]:
size(peak_period)

(10296,)

In [5]:
# Same as for the hourly calls; but this is for individual emergency calls
# inc_peak_period = (calls[!,:hour] .>= 8) .* (calls[!,:hour] .<= 20)
# inc_indices = 1:DataFrames.nrow(calls);

# inc_train_filter = (calls[!,:year] .== 2019) .* (calls[!,:month] .<= 3)
# inc_test_filter  = .~inc_train_filter

# inc_train_indices = inc_indices[inc_train_filter]
# inc_test_indices = inc_indices[inc_test_filter];

In [6]:
# we distinguish between peak and offpeak hours
train_peak = indices[peak_period .* train_filter]
train_offpeak = indices[.~peak_period .* train_filter]

test_peak = indices[peak_period .* test_filter]
test_offpeak = indices[.~peak_period .* test_filter]

p = DeploymentProblem(30, length(locations), length(regions), demand, train_indices,
      test_indices, coverage[regions,:], Array{Bool,2}(adjacent));

#test_inc_peak = inc_peak_period .* inc_test_filter
#test_inc_offpeak = .~inc_peak_period .* inc_test_filter;

In [21]:
#declare data structures to record results
scenarios = Dict{Symbol, Dict{Int, Vector{Vector{Int}}}}()
generated_deployment = Dict{Symbol, Dict{Int, Vector{Vector{Int}}}}()
upperbounds = Dict{Symbol, Dict{Int, Vector{Float64}}}()
lowerbounds = Dict{Symbol, Dict{Int, Vector{Float64}}}()
upptiming = Dict{Symbol, Dict{Int, Vector{Float64}}}()
lowtiming = Dict{Symbol, Dict{Int, Vector{Float64}}}()
amb_deployment = Dict{Symbol, Dict{Int, Vector{Int}}}()

Dict{Symbol,Dict{Int64,Array{Int64,1}}}()

In [25]:
(deployment_model, name) = (next_dp -> RobustDeployment(next_dp, α= .1), :Robust01)
amb_deployment[name] = Dict{Int, Vector{Int}}()
scenarios[name] = Dict{Int, Vector{Vector{Int}}}()
generated_deployment[name] = Dict{Int, Vector{Vector{Int}}}()
upperbounds[name] = Dict{Int, Vector{Float64}}()
lowerbounds[name] = Dict{Int, Vector{Float64}}()
upptiming[name] = Dict{Int, Vector{Float64}}()
lowtiming[name] = Dict{Int, Vector{Float64}}()
namb = 40
println("$namb ")
p.nambulances = namb

#AmbulanceDeployment.Gamma(p,.01)

model = deployment_model(p)
# set_optimizer(model.m, Gurobi.Optimizer)
# @time AmbulanceDeployment.optimize_robust!(model, p);
# print(deployment(model))

# amb_deployment[name][namb] = deployment(model)

# # for tracking purposes
# scenarios[name][namb] = model.scenarios
# generated_deployment[name][namb] = model.deployment
# upperbounds[name][namb] = model.upperbounds
# lowerbounds[name][namb] = model.lowerbounds
# upptiming[name][namb] = model.upptiming
# lowtiming[name][namb] = model.lowtiming

40 


LoadError: [91mUndefVarError: paramss not defined[39m

In [115]:
print(sum(model.scenarios[1])) #why are we testing on 299 test calls???

294

# Cross validation

In [107]:
using DelimitedFiles
alpha_list = [.1,.05,.01,.001,.0001]
for x in alpha_list
    (deployment_model, name) = (next_dp -> RobustDeployment(next_dp, α= x), :Robust01)
    amb_deployment[name] = Dict{Int, Vector{Int}}()
    scenarios[name] = Dict{Int, Vector{Vector{Int}}}()
    generated_deployment[name] = Dict{Int, Vector{Vector{Int}}}()
    upperbounds[name] = Dict{Int, Vector{Float64}}()
    lowerbounds[name] = Dict{Int, Vector{Float64}}()
    upptiming[name] = Dict{Int, Vector{Float64}}()
    lowtiming[name] = Dict{Int, Vector{Float64}}()
    namb = 40
    println("$namb ")
    p.nambulances = namb

    model = deployment_model(p)
    set_optimizer(model.m, Gurobi.Optimizer)
    @time AmbulanceDeployment.optimize_robust!(model, p);
    print(deployment(model))

    amb_deployment[name][namb] = deployment(model)

    # for tracking purposes
    scenarios[name][namb] = model.scenarios
    generated_deployment[name][namb] = model.deployment
    upperbounds[name][namb] = model.upperbounds
    lowerbounds[name][namb] = model.lowerbounds
    upptiming[name][namb] = model.upptiming
    lowtiming[name][namb] = model.lowtiming

    #save julia variable as .csv
    deployment(model)
    #or
    x_save = Array{Int64}(undef, 44)
    count = 1
    for i in JuMP.value.(model.x)
        x_save[count] = convert(Int,i)
        count = count + 1
    end


    writedlm( "robust" * string(x) * ".csv",  x_save, ',') #it writes to pwd(), ambulance_legacy
end

40 
Academic license - for non-commercial use only - expires 2022-07-26
Academic license - for non-commercial use only - expires 2022-07-26


┌ Warning: Passing optimizer attributes as keyword arguments to
│ Gurobi.Optimizer is deprecated. Use
│     MOI.set(model, MOI.RawParameter("key"), value)
│ or
│     JuMP.set_optimizer_attribute(model, "key", value)
│ instead.
└ @ Gurobi C:\Users\Owner\.julia\packages\Gurobi\BAtIN\src\MOI_wrapper\MOI_wrapper.jl:273
┌ Warning: Passing optimizer attributes as keyword arguments to
│ Gurobi.Optimizer is deprecated. Use
│     MOI.set(model, MOI.RawParameter("key"), value)
│ or
│     JuMP.set_optimizer_attribute(model, "key", value)
│ instead.
└ @ Gurobi C:\Users\Owner\.julia\packages\Gurobi\BAtIN\src\MOI_wrapper\MOI_wrapper.jl:273


Academic license - for non-commercial use only - expires 2022-07-26
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 508 rows, 9450 columns and 27057 nonzeros
Model fingerprint: 0x9fb4eda8
Variable types: 1 continuous, 9449 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [4e+01, 4e+01]
  RHS range        [1e+00, 4e+01]

User MIP start did not produce a new incumbent solution
User MIP start violates constraint R23 by 1.000000000

Found heuristic solution: objective 2.000000e+09
Presolve removed 354 rows and 7112 columns
Presolve time: 0.02s
Presolved: 154 rows, 2338 columns, 4615 nonzeros
Found heuristic solution: objective 258.0000400
Variable types: 0 continuous, 2338 integer (1421 binary)

Root relaxation: cutoff, 314 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds  

┌ Warning: Passing optimizer attributes as keyword arguments to
│ Gurobi.Optimizer is deprecated. Use
│     MOI.set(model, MOI.RawParameter("key"), value)
│ or
│     JuMP.set_optimizer_attribute(model, "key", value)
│ instead.
└ @ Gurobi C:\Users\Owner\.julia\packages\Gurobi\BAtIN\src\MOI_wrapper\MOI_wrapper.jl:273
┌ Warning: Passing optimizer attributes as keyword arguments to
│ Gurobi.Optimizer is deprecated. Use
│     MOI.set(model, MOI.RawParameter("key"), value)
│ or
│     JuMP.set_optimizer_attribute(model, "key", value)
│ instead.
└ @ Gurobi C:\Users\Owner\.julia\packages\Gurobi\BAtIN\src\MOI_wrapper\MOI_wrapper.jl:273


Academic license - for non-commercial use only - expires 2022-07-26
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 508 rows, 9450 columns and 27057 nonzeros
Model fingerprint: 0xa2d857de
Variable types: 1 continuous, 9449 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [4e+01, 4e+01]
  RHS range        [1e+00, 4e+01]

User MIP start did not produce a new incumbent solution
User MIP start violates constraint R23 by 1.000000000

Found heuristic solution: objective 2.000000e+09
Presolve removed 353 rows and 7067 columns
Presolve time: 0.02s
Presolved: 155 rows, 2383 columns, 4704 nonzeros
Found heuristic solution: objective 259.0000400
Variable types: 0 continuous, 2383 integer (1464 binary)

Root relaxation: cutoff, 313 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds  

...

Academic license - for non-commercial use only - expires 2022-07-26
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 508 rows, 9450 columns and 27057 nonzeros
Model fingerprint: 0xa2d857de
Variable types: 1 continuous, 9449 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [4e+01, 4e+01]
  RHS range        [1e+00, 4e+01]

User MIP start did not produce a new incumbent solution
User MIP start violates constraint R23 by 1.000000000

Found heuristic solution: objective 2.000000e+09
Presolve removed 353 rows and 7067 columns
Presolve time: 0.02s
Presolved: 155 rows, 2383 columns, 4704 nonzeros
Found heuristic solution: objective 259.0000400
Variable types: 0 continuous, 2383 integer (1464 binary)

Root relaxation: cutoff, 313 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds  

┌ Warning: Passing optimizer attributes as keyword arguments to
│ Gurobi.Optimizer is deprecated. Use
│     MOI.set(model, MOI.RawParameter("key"), value)
│ or
│     JuMP.set_optimizer_attribute(model, "key", value)
│ instead.
└ @ Gurobi C:\Users\Owner\.julia\packages\Gurobi\BAtIN\src\MOI_wrapper\MOI_wrapper.jl:273
┌ Warning: Passing optimizer attributes as keyword arguments to
│ Gurobi.Optimizer is deprecated. Use
│     MOI.set(model, MOI.RawParameter("key"), value)
│ or
│     JuMP.set_optimizer_attribute(model, "key", value)
│ instead.
└ @ Gurobi C:\Users\Owner\.julia\packages\Gurobi\BAtIN\src\MOI_wrapper\MOI_wrapper.jl:273


Academic license - for non-commercial use only - expires 2022-07-26
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 508 rows, 9450 columns and 27057 nonzeros
Model fingerprint: 0xa2d857de
Variable types: 1 continuous, 9449 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [4e+01, 4e+01]
  RHS range        [1e+00, 4e+01]

User MIP start did not produce a new incumbent solution
User MIP start violates constraint R23 by 1.000000000

Found heuristic solution: objective 2.000000e+09
Presolve removed 353 rows and 7067 columns
Presolve time: 0.02s
Presolved: 155 rows, 2383 columns, 4704 nonzeros
Found heuristic solution: objective 259.0000400
Variable types: 0 continuous, 2383 integer (1464 binary)

Root relaxation: cutoff, 313 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds  

┌ Warning: Passing optimizer attributes as keyword arguments to
│ Gurobi.Optimizer is deprecated. Use
│     MOI.set(model, MOI.RawParameter("key"), value)
│ or
│     JuMP.set_optimizer_attribute(model, "key", value)
│ instead.
└ @ Gurobi C:\Users\Owner\.julia\packages\Gurobi\BAtIN\src\MOI_wrapper\MOI_wrapper.jl:273
┌ Warning: Passing optimizer attributes as keyword arguments to
│ Gurobi.Optimizer is deprecated. Use
│     MOI.set(model, MOI.RawParameter("key"), value)
│ or
│     JuMP.set_optimizer_attribute(model, "key", value)
│ instead.
└ @ Gurobi C:\Users\Owner\.julia\packages\Gurobi\BAtIN\src\MOI_wrapper\MOI_wrapper.jl:273


Academic license - for non-commercial use only - expires 2022-07-26
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 508 rows, 9450 columns and 27057 nonzeros
Model fingerprint: 0xa2d857de
Variable types: 1 continuous, 9449 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [4e+01, 4e+01]
  RHS range        [1e+00, 4e+01]

User MIP start did not produce a new incumbent solution
User MIP start violates constraint R23 by 1.000000000

Found heuristic solution: objective 2.000000e+09
Presolve removed 353 rows and 7067 columns
Presolve time: 0.02s
Presolved: 155 rows, 2383 columns, 4704 nonzeros
Found heuristic solution: objective 259.0000400
Variable types: 0 continuous, 2383 integer (1464 binary)

Root relaxation: cutoff, 313 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds  